In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import *
from constants import *
from imports import *
from loaders import *
from train_utils import *
from rollout import RwEvaluator

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

('1.13.1', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'2vb9g986'

In [3]:
wandb.init(id='fszwymsu', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) 
m.train()
sum([torch.numel(p) for p in m.parameters()]) / 1000

30484.11

In [5]:
m = try_load_state_dict(m, torch.load(f"{BESPOKE_ROOT}/models/m.torch"))

In [6]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(m, wandb=wandb)

CPU times: user 19 µs, sys: 9 µs, total: 28 µs
Wall time: 33.6 µs


In [7]:
model_stem = "2.22"

In [8]:
%%time

rnn_only = True
rnn_is_first = False

if rnn_only: # m cnn backbone needs to be all loaded up and ready

    if rnn_is_first:
        freeze_model(m, True)
        freeze_model(m.rnn, False)
        m.hidden_init.requires_grad = True
        m.cell_init.requires_grad = True
    else:
        freeze_model(m.backbone, True)
        
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 42 #21 #7 # these have to be multiple of Zloader's internal bptt, currently 3 for bs 32 or 7 for bs 16
    seqlen = bptt*10 #5 #15 
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    n_workers = 5 if seqlen > 100 else 4 if seqlen > 400 else 8 # maxxing out ram here w higher bs. Was 8 w bs 16
    # fewer than five workers we start to have to wait too long for chunk. More is better here.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = 0 #.9 TODO temp disabled
    n_workers = 8
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
    
lr = 3e-4
m.use_rnn = rnn_only

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been depreca

waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk
waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
CPU times: user 6.87 s, sys: 57.6 s, total: 1min 4s
Wall time: 3min 41s


Process Process-2:
Process Process-4:
Process Process-6:
Process Process-3:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process Process-5:
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 117, in keep_chunk_filled
    timings = fill_chunk_inplace(shm_img, shm_aux, shm_targets, constant_seq_aug=constant_seq_aug)
  File "/home/beans/bespoke/loaders.py", line 105, in fill_chunk_inplace
    t.join()
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 1011, in join
    self._wait_for_tstate_lock()
Traceback (most recent call last):
  File "/home/beans/minicond

In [9]:
opt = torch.optim.Adam(m.parameters(), lr=lr)
#opt = torch.optim.AdamW(m.parameters(), lr=3e-4, weight_decay=.001)

In [10]:
m.backbone.training, m.training

(False, True)

In [11]:
[(n,p.requires_grad) for (n,p) in m.named_parameters()]

[('hidden_init', True),
 ('cell_init', True),
 ('backbone.conv_stem.weight', False),
 ('backbone.bn1.weight', False),
 ('backbone.bn1.bias', False),
 ('backbone.blocks.0.0.conv_dw.weight', False),
 ('backbone.blocks.0.0.bn1.weight', False),
 ('backbone.blocks.0.0.bn1.bias', False),
 ('backbone.blocks.0.0.se.conv_reduce.weight', False),
 ('backbone.blocks.0.0.se.conv_reduce.bias', False),
 ('backbone.blocks.0.0.se.conv_expand.weight', False),
 ('backbone.blocks.0.0.se.conv_expand.bias', False),
 ('backbone.blocks.0.0.conv_pw.weight', False),
 ('backbone.blocks.0.0.bn2.weight', False),
 ('backbone.blocks.0.0.bn2.bias', False),
 ('backbone.blocks.0.1.conv_dw.weight', False),
 ('backbone.blocks.0.1.bn1.weight', False),
 ('backbone.blocks.0.1.bn1.bias', False),
 ('backbone.blocks.0.1.se.conv_reduce.weight', False),
 ('backbone.blocks.0.1.se.conv_reduce.bias', False),
 ('backbone.blocks.0.1.se.conv_expand.weight', False),
 ('backbone.blocks.0.1.se.conv_expand.bias', False),
 ('backbone.block

In [12]:
%%time
"""m.model_stem = model_stem
rw_evaluator.evaluate()"""

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.5 µs


'm.model_stem = model_stem\nrw_evaluator.evaluate()'

In [13]:
for n, mm in m.backbone.named_modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .99 #.4

In [14]:
trainer = Trainer(dataloader_trn, m, model_stem=model_stem, opt=opt, wandb=wandb, 
                  rw_evaluator=rw_evaluator, rnn_only=rnn_only)

In [15]:
trainer.reload_state()

Loading model from /home/beans/bespoke/models/m.torch. 
Last modified 81 min ago.
Loading opt from /home/beans/bespoke/models/opt.torch. 
Last modified 81 min ago.
Currently at epoch 56.


In [16]:
copy_cnn_params = False
if copy_cnn_params:
    m.copy_cnn_params_to_rnn() # when first trn rnn, has to be after reload_state.
    print("copied")
    
if rnn_only:
    trainer.loss_manager.update_emas = False
    trainer.loss_manager.loss_emas['te'] = .002 # manual. Bc cnn doesn't cache this.

In [17]:
trainer.loss_manager.update_emas

False

In [18]:
trainer.loss_manager.loss_weights

{'has_stop': 0.1,
 'stop_dist': 0.1,
 'has_lead': 0.1,
 'lead_dist': 0.1,
 'lead_speed': 0.02,
 'lane_width': 0.02,
 'dagger_shift': 0.05,
 'rd_is_lined': 0.02,
 'left_turn': 0.01,
 'right_turn': 0.01,
 'td': 0.03,
 'pitch': 0.1,
 'yaw': 0.1,
 'unc': 0.1,
 'te': 0.05,
 'wp_angles': 1,
 'wp_angles_i': 0.5,
 'wp_curvatures': 0.2,
 'wp_curvatures_i': 0.1,
 'wp_headings': 0.02,
 'wp_headings_i': 0.01,
 'wp_rolls': 0.05,
 'wp_rolls_i': 0.025,
 'wp_zs': 0.02,
 'wp_zs_i': 0.01}

In [19]:
trainer.loss_manager.loss_emas

{'has_stop': 0.007291453775047122,
 'stop_dist': 0.0010972422739921955,
 'has_lead': 0.005333186583720615,
 'lead_dist': 0.0005156506595824284,
 'lead_speed': 0.022654750599122395,
 'lane_width': 0.04776586065100641,
 'dagger_shift': 0.036571414587329036,
 'rd_is_lined': 0.0005849007004490738,
 'left_turn': 0.0012587523737711843,
 'right_turn': 0.0009495396139817013,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.2182893118458162,
 'te': 0.002,
 'wp_angles': 0.09571978783798096,
 'wp_angles_i': 0.28696715106495974,
 'wp_curvatures': 0.7270083407715314,
 'wp_curvatures_i': 1.5582720477174539,
 'wp_headings': 0.11876611384250395,
 'wp_headings_i': 0.3882927555227806,
 'wp_rolls': 3.0530489087143833,
 'wp_rolls_i': 0.002468118874703614,
 'wp_zs': 3.751640680590813,
 'wp_zs_i': 0.050447209129057854}

In [20]:
trainer.train()

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


157.25 50.53125 104.8125 11.6171875 19.484375 14.734375 3.259765625 480.5 0.65185546875 176.5
wp_angles_i tensor(0.0576, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.0552, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(0.1689, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0027, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0259, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.1201, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1421, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(1.6719, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(1.5107, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(5.5469, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te tensor(0.00

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning:

None of the inputs have requires_grad=True. Gradients will be None



36.5625 5000.0 1986.0 3.1953125 246.625 1.77734375 12.984375 163.875 2.248046875 515.5
Waiting for batch...
Waiting for batch...
Waiting for batch...
Waiting for batch...
112.8125 0.0 0.0 23.625 0.0 9.859375 0.0 769.0 0.0 317.25
393.75 478.5 87.875 36.4375 12.0859375 50.53125 26.421875 216.25 0.1864013671875 409.5
Waiting for batch...
Waiting for batch...
347.25 306.25 26.859375 5.7265625 7.26171875 4.59765625 13.3515625 272.0 0.34765625 34.15625
wp_angles_i tensor(0.0215, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.0442, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(0.8677, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0006, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0903, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0484, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_heading

Waiting for batch...
482.0 3888.0 309.0 21.015625 263.0 7.46484375 2.953125 318.5 0.1697998046875 726.5
wp_angles_i tensor(1.1797, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(1.0664, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(4.1758, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0005, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0183, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0592, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.0521, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.4707, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(5.9727, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(4.5352, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te t

...

Waiting for batch...
Waiting for batch...
Waiting for batch...
362.0 17.640625 23.84375 41.96875 9.765625 11.8671875 3.0 150.875 0.0421142578125 215.625
wp_angles_i tensor(0.0209, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.0469, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(0.0578, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(7.0035e-05, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0032, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0363, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.0640, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.7578, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(2.1250, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(3.4922, device=

lead_speed tensor(0., device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
dagger_shift tensor(0.0092, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
lane_width tensor(0.0840, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
rd_is_lined tensor(5.1260e-06, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
left_turn tensor(0.0002, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
right_turn tensor(1.1921e-06, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
unc tensor(1.4346, device='cuda:0', dtype=torch.float16,
       grad_fn=<MseLossBackward0>)
Waiting for batch...
Waiting for batch...
131.25 2028.0 32.53125 24.53125 26.484375 4.78515625 5.5625 49.9375 17.609375 1288.0
Waiting for batch...
Waiting for batch...


 {'logistical/obs_consumed_per_second': 3.796875, 'logistical/obs_generated_per_second': 23.8515625, 'logistical/slowest_runner_obs_per_sec': 3.0, 'logistical/data_consumption_ratio':

Waiting for batch...
loss is nan for wp_curvatures. Skipping
Waiting for batch...
221.25 1920.0 179.5 23.4375 33.875 4.109375 0.63623046875 85.0625 0.152587890625 245.125
Waiting for batch...
waiting for chunk
Waiting for batch...
Waiting for batch...
5.9140625 653.5 39.0625 1.603515625 3.08984375 3.068359375 11.703125 212.25 0.45166015625 80.0625
Waiting for batch...
Waiting for batch...
39744.0 47040.0 27120.0 42.4375 2168.0 18.296875 45.21875 341.5 3.5390625 74.5625
Waiting for batch...
Waiting for batch...
Waiting for batch...
691.0 47936.0 17152.0 54.53125 506.25 1.8232421875 70.375 158.0 7.18359375 864.0
Waiting for batch...


 {'logistical/obs_consumed_per_second': 3.765625, 'logistical/obs_generated_per_second': 24.6640625, 'logistical/slowest_runner_obs_per_sec': 2.9453125, 'logistical/data_consumption_ratio': 0.15050093, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 4.90743334, 'timing/model forward': 0.00656191, 'timing/calc losses': 0.03004391, '

Waiting for batch...
Waiting for batch...
17.375 1553.0 14552.0 1.947265625 448.5 9.21875 11.0703125 1229.0 1.4453125 53.03125
Waiting for batch...
Waiting for batch...
Waiting for batch...
210.375 273.0 225.0 7.5859375 32.53125 1.466796875 4.8203125 158.25 0.201171875 451.25
Waiting for batch...
Waiting for batch...
29.375 1669.0 284.75 5.5859375 142.25 3.265625 8.4140625 90.9375 0.191162109375 765.0
Waiting for batch...
Waiting for batch...
62.40625 42.03125 60.4375 15.71875 2.44140625 3.701171875 40.125 302.0 3.0 93.6875
wp_angles_i tensor(0.0060, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.0754, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(0.0840, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0043, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.1384, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.

{'logistical/obs_consumed_per_second': 1.0, 'logistical/obs_generated_per_second': 26.0, 'logistical/slowest_runner_obs_per_sec': 3.0, 'logistical/data_consumption_ratio': 0.0303897, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 13.71407485, 'timing/model forward': 0.00315237, 'timing/calc losses': 0.0097847, 'timing/backwards': 0.02441335, 'timing/get worst': 7.15e-06, 'timing/logging': 6.79065752, 'timing/calc timing': 0.03171229, 'timing/trn update': 20.57381225}
6.4921875 11176.0 24976.0 3.890625 339.75 8.4140625 38.8125 392.5 1.447265625 695.5
Waiting for batch...
Waiting for batch...
38.84375 1767.0 64.75 28.0 62.25 12.71875 31.609375 421.0 1.201171875 185.375
wp_angles_i tensor(0.6035, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.4065, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(2.9609, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0

In [22]:
t = torch.randn(3,4,5)
t[0,0,0] = torch.inf

In [23]:
t

tensor([[[    inf, -0.7854, -1.3716,  0.8498, -0.0986],
         [ 0.3141, -1.5963, -0.3781, -1.1632,  0.0405],
         [-0.0689, -0.9696,  1.4334,  1.0205,  2.2483],
         [ 1.2481, -0.8255, -0.5951, -0.0256,  0.2744]],

        [[ 0.6463,  1.3897,  0.8417, -0.3971, -1.4846],
         [ 0.3036, -0.4071, -0.0882, -0.6457,  0.2220],
         [ 1.5495,  0.6217, -0.4204,  0.3186,  0.4908],
         [-2.7397, -2.3005,  1.9402, -0.5320, -1.1312]],

        [[ 0.2612,  0.7503, -0.6469,  0.5444, -0.6493],
         [-1.0282, -1.1678, -0.1762,  1.1714,  0.2500],
         [ 0.6518, -0.5325, -1.2205,  0.8746,  0.5583],
         [ 0.3824, -0.7756,  0.8413,  0.3293, -1.3277]]])

In [24]:
t.max()

tensor(inf)

In [20]:
AUX_TARGET_PROPS

['has_stop',
 'stop_dist',
 'has_lead',
 'lead_dist',
 'lead_speed',
 'dagger_shift',
 'lane_width',
 'rd_is_lined',
 'left_turn',
 'right_turn']

In [20]:
trainer.loss_manager.loss_emas

{'has_stop': 0.6581827152218972,
 'stop_dist': 0.1914074304465102,
 'has_lead': 0.6045025000545183,
 'lead_dist': 0.18079276826857715,
 'lead_speed': 0.8834656769419045,
 'lane_width': 4.096574170043749,
 'dagger_shift': 2.512469867257369,
 'rd_is_lined': 0.10581200184235365,
 'left_turn': 0.08856740587054233,
 'right_turn': 0.07401701428385941,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 117.34792707051348,
 'te': 0.002,
 'wp_angles': 0.0859911379087128,
 'wp_angles_i': 4.15318218310992,
 'wp_curvatures': 0.7827851002866839,
 'wp_curvatures_i': 17.866540970943753,
 'wp_headings': 0.12228421566478304,
 'wp_headings_i': 2.901144808827211,
 'wp_rolls': 4.211842307760256,
 'wp_rolls_i': 0.13459417300430104,
 'wp_zs': 5.043518468202303,
 'wp_zs_i': 1.1309414719989206}

In [20]:
m.hidden_init, m.cell_init

(Parameter containing:
 tensor([[[ 0.0015, -0.0488, -0.0292,  ..., -0.0560, -0.0283,  0.0664]]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[[-0.3869,  0.0330,  0.0708,  ..., -0.0204,  0.3181, -0.2530]]],
        device='cuda:0', requires_grad=True))

In [21]:
m.training, m.backbone.training 

(True, False)

In [22]:
set_trainer_should_stop(False)

In [28]:
{k:trainer.loss_manager.loss_emas["wp_angles"]/trainer.loss_manager.loss_emas[k] for k,v in trainer.loss_manager.loss_emas.items()}

{'has_stop': 29.734797363340526,
 'stop_dist': 87.87280383313023,
 'has_lead': 40.201898525296386,
 'lead_dist': 127.76564504863305,
 'lead_speed': 0.34846566491395553,
 'lane_width': 3.809900606793288,
 'dagger_shift': 4.134576796863424,
 'rd_is_lined': 16457.381372040072,
 'left_turn': 125.55969474655697,
 'right_turn': 283.74133223646584,
 'td': 0.015300805875045518,
 'pitch': 0.015300805875045518,
 'yaw': 0.015300805875045518,
 'unc': 0.11512757866420895,
 'te': 61.54656219577027,
 'wp_angles': 1.0,
 'wp_angles_i': 0.240031243936724,
 'wp_curvatures': 0.16382346075192508,
 'wp_curvatures_i': 0.003522151816451157,
 'wp_headings': 0.5183519273228973,
 'wp_headings_i': 0.005565440760627469,
 'wp_rolls': 0.027721036050055964,
 'wp_rolls_i': 8.663331955712781,
 'wp_zs': 0.00020285505128895626,
 'wp_zs_i': 0.0035411505624853203}

In [23]:
{'has_stop': 0.005145757574225003,
 'stop_dist': 0.001741244754645775,
 'has_lead': 0.0038059908701619494,
 'lead_dist': 0.0011975680840668381,
 'lead_speed': 0.4390907746627964,
 'lane_width': 0.04016064316156499,
 'dagger_shift': 0.0370069456362572,
 'rd_is_lined': 9.297229935401817e-06,
 'left_turn': 0.0012186080816722507,
 'right_turn': 0.0005392519219686348,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.329030459302299,
 'te': 0.002486053701322257,
 'wp_angles': 0.15300805875045517,
 'wp_angles_i': 0.6374505928519476,
 'wp_curvatures': 0.9339813604728601,
 'wp_curvatures_i': 1.44164213359285,
 'wp_headings': 0.2951818073498582,
 'wp_headings_i': 1.49253209788985,
 'wp_rolls': 5.519564942456264,
 'wp_rolls_i': 0.017661571729288113,
 'wp_zs': 754.2728553133405,
 'wp_zs_i': 43.208571917673005


import sys
sys.getsizeof(trainer.loss_manager.loss_emas)

1176

In [16]:
torch.save(trainer.model.state_dict(), f"{BESPOKE_ROOT}/models/m{trainer.model_stem}_e{trainer.current_epoch}.torch")

In [16]:
m.training

Error in callback <function _WandbInit._resume_backend at 0x7f62ef023160> (for pre_run_cell):


Exception: The wandb backend process has shutdown

True

Error in callback <function _WandbInit._pause_backend at 0x7f62e3b863a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
trainer.opt.param_groups[0]['lr'] = 3e-4

In [28]:
snr

0.15067877149872744



 {'logistical/obs_consumed_per_second': 38.2265625, 'logistical/obs_generated_per_second': 85.1328125, 'logistical/slowest_runner_obs_per_sec': 5.3125, 'logistical/data_consumption_ratio': 0.45004667, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00400971, 'timing/model forward': 0.19439457, 'timing/calc losses': 0.00539116, 'timing/backwards': 1.11790375, 'timing/get worst': 0.00264816, 'timing/logging': 0.01570235, 'timing/calc timing': 0.01359244, 'timing/trn update': 1.35364565, 'wp_angles': 0.01003864, 'wp_curvatures': 0.04547036, 'wp_headings': 0.01048879, 'wp_rolls': 0.52840137, 'wp_zs': 0.09037566, 'has_stop': 0.01459547, 'stop_dist': 0.00825622, 'has_lead': 0.01966284, 'lead_dist': 0.00662124, 'lead_speed': 16.11909485, 'dagger_shift': 0.14152849, 'lane_width': 0.12975717, 'rd_is_lined': 0.01347083, 'pitch': 4.037e-05, 'yaw': 4.98e-05, 'unc': 1.57641983, 'logistical/max_param': 32.69097137, 'logistical/lr': 0.0006, 'logistical/mins_since_slowest

In [9]:
%%time

rnn_only = True

if rnn_only: # m cnn backbone needs to be all loaded up and ready
    #freeze_model(m, True)
    #unfreeze_part_of_model(m, "rnn")
    freeze_model(m.backbone, True)
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 14 #35 #8 #7
    seqlen = bptt*5 # needs to be mult of 7
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = .9
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug)

Launching 3 loader workers


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated.

Got first chunk
CPU times: user 1.1 s, sys: 5.72 s, total: 6.83 s
Wall time: 45.2 s


Process Process-3:
Process Process-4:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 133, in make_chunks
    time.sleep(.1)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 1

In [25]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
